# **Auxilliary**

In [1]:
# Library Imports
import sys
import numpy as np
import pandas as pd

from dataset import dataset_generator
from bot.bot import Bot
from modelling import data_preparation as dp
from modelling import model_trainer as mt

path = "binary-options-research/"
if path not in sys.path:
    sys.path.append(path)
sys.stdout = open("stdout", 'w')

np.random.seed(42)

%load_ext autoreload
%autoreload 2

# **Data Generation**

In [11]:
# Dataset Generation
dataset_generator.make(["BTCUSDT"], "3m")

# **Data Extraction**

In [14]:
# Data Extraction
candles = pd.read_csv(f"dataset/BTCUSDT-3m-2020_2024.csv", index_col=0)
candles_training = candles[(candles['open_time'] >= '2020-01-01') & (candles['open_time'] < '2023-01-01')].reset_index(drop=True)

bot = Bot(candles_training, 0, None, None)
bot._pattern_features_service(1)

# **Model Training**

In [18]:
# Save data Double Bottom
features = pd.read_csv("features/features_3m_30.csv")
features = features[features['pattern'] == "Double Bottom"].reset_index(drop=True)
X_train, y_train, X_valid, y_valid, X_test, y_test = dp.split_data(features)
print("Writing data")
dp.save_data(X_train, y_train, X_valid, y_valid, X_test, y_test)

In [19]:
# Parameter Hypertuning Double Bottom
X_train, y_train, X_valid, y_valid, X_test, y_test = dp.load_data()
mt.find_hyper_params(X_train, y_train, X_valid, y_valid)

In [20]:
# Model Training Double Bottom
hyper = {'base_score': 2, 'colsample_bylevel': 10, 'colsample_bynode': 7, 'colsample_bytree': 9, 'gamma': 2, 'learning_rate': 6, 'max_depth': 14, 'min_child_weight': 0, 'n_estimators': 4, 'reg_alpha': 0, 'reg_lambda': 2, 'scale_pos_weight': 1, 'subsample': 4}
X_train, y_train, X_valid, y_valid, X_test, y_test  = dp.load_data()
mt.fit_model(X_train, y_train, X_valid, y_valid, X_test, y_test, hyper, "DBmodel_3m_30") 

In [21]:
# Save data Double Top
features = pd.read_csv("features/features_3m_30.csv")
features = features[features['pattern'] == "Double Top"].reset_index(drop=True)
X_train, y_train, X_valid, y_valid, X_test, y_test = dp.split_data(features)
print("Writing data")
dp.save_data(X_train, y_train, X_valid, y_valid, X_test, y_test)

In [22]:
# Parameter Hypertuning Double Top
X_train, y_train, X_valid, y_valid, X_test, y_test = dp.load_data()
mt.find_hyper_params(X_train, y_train, X_valid, y_valid)

In [23]:
# Model Training Double Top
hyper = {'base_score': 9, 'colsample_bylevel': 8, 'colsample_bynode': 2, 'colsample_bytree': 6, 'gamma': 0, 'learning_rate': 9, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 1, 'reg_alpha': 4, 'reg_lambda': 2, 'scale_pos_weight': 5, 'subsample': 1}
X_train, y_train, X_valid, y_valid, X_test, y_test  = dp.load_data()
mt.fit_model(X_train, y_train, X_valid, y_valid, X_test, y_test, hyper, "DTmodel_3m_30") 

# **Forward Testing**

In [25]:
# Model testing
candles = pd.read_csv(f"dataset/BTCUSDT-3m-2020_2024.csv", index_col=0)
candles_testing = candles[(candles['open_time'] >= '2023-01-01')].reset_index(drop=True)

bot = Bot(candles_testing, 1, "DBmodel_3m_30", "DTmodel_3m_30")
bot._pattern_features_service(30)